## Motivation

Proper audience targeting is a critical component of a successful advertising campaign. This study aims to identify customer demographics that may be particularly receptive to Game Revenant's (GR) advertising, specifically to optimize social media-based campaigns. 

Due to their large reach and popularity, Facebook, Instagram, and Twitter can be an effective means of reaching new audiences. Facebook (which own Instagram) and Twitter both offer APIs that allow businesses to target their advertisements based on a number of demographic features, such as age, gender, language, location, and interests. This project aims to inform audience targeting based on interests. 

Analysis is performed on unstructured social media data, specifically from Twitter. Compared to other social media, Twitter's API allows greater access to user data, enabling more effective data mining. GR currently has too few followers on Twitter to allow for productive data mining. However, the Twitter accounts of rival companies can be analyzed as well. *Where Shadows Slumber* (WSS) is frequently compared to *Monument Valley*, a mobile puzzle game produced by USTWO Games, by game critics and customer reviewers. As of this writing, @ustwogames has over 126k followers. 

Customer interests were estimated by 1) examining the profile description of @ustogames followers, and 2) examining the most popular friends among @ustogames followers. Preprocessed descriptions were examined via two clustering approches, K-modes and 
DBSCAN. In Twitter's official terminology, a 'friend' is an account a user follows. The most popular friends were clustered via K-modes. The rationale behind the unorthodox application of K-modes will be explained later in the notebook. 


## Clustering methods and results

### Data exploration

The Twitter profiles of @ustogames were mined via the python package Tweepy and written to a SQL database (refer to *pull_data_twitter.py* and *sqlite_fx.py* for script and function codes). 

In [2]:
# Autoreload to accomodate script updates without restarting notebook
%load_ext autoreload
%autoreload 2


C:\Users\Vincent S\game-revenant\Customer-Segmentation


In [164]:
import pandas as pd
import sqlalchemy as sa
from pathlib import Path

# Move to main directory of the Customer-Segmentation project
%cd ..

DB_NAME = 'customer-segmentation'
TAB_NAME = 'ustwo_followers'

# Pull interim data from SQLite DB
e = sa.create_engine('sqlite:///./data/interim/' + DB_NAME + '.sqlite')
query = 'SELECT * FROM ' + TAB_NAME
users = pd.read_sql_query(query, e)

users.head()

,id,id_string,name,screen_name,location,url,description,protected,verified,followers_count,friend_count,listed_count,favourites_count,statuses_count,created_at,default_profile,default_profile_image
0,1114109046792032256,1114109046792032256,Hiyoru,Hiyoru6,None,None,so para fotos de desenhos,0,0,1,21,0,139,5,Fri Apr 05 10:14:30 +0000 2019,1,0
1,1120790192128954371,1120790192128954371,Tom Baines 🏳️‍🌈 🇪🇺,TomBaines16,"North West, England",None,Extreme sports calendar model. \nKeeping retro alive since 1988.,0,0,29,169,0,961,456,Tue Apr 23 20:42:59 +0000 2019,0,0
2,113652889,113652889,Marmalade Games,MarmaladeGames,"London, UK",http://t.co/sPUf5LShHE,None,0,0,506,262,13,304,471,Fri Feb 12 15:07:07 +0000 2010,0,0
3,363116063,363116063,MIT SHAH,mitshah97,Ahmedabad,None,#Unity3d #Game #Developer,0,0,39,279,1,22,13,Sat Aug 27 15:13:29 +0000 2011,1,0
4,972169508,972169508,Kyrie E.H.C.,KyrieEHC,"Madison, WI",https://t.co/ajE03P8x2g,"Still believes in the warmth in interaction. Studied games in culture & learning at MIT in CMS, SM '16. Works at UW-Ex; personal account here. She/her.",0,0,333,1103,18,1118,734,Mon Nov 26 15:27:59 +0000 2012,0,0


In [4]:
print(str(len(users)) + ' @ustwogames follower profiles were mined')

125925 @ustwogames follower profiles were mined


### Text wrangling of profile descriptions

Although a number of features were mined, only the profile description and identifying user ID are of interest in the proceeding analysis. 

Prior to analysis, unstructured text data must be preprocessed. Punctuation, emojis, and excess white space were removed. All words were converted to lowercase and tokenized. Stopwords were removed. The NLTK stopword list was extended by several words that were found to be common in profile descriptions, but did not add any useful information about user interests. 

In [166]:
import nltk
import re
import numpy as np

def normalize_doc(doc, stop_words):
    ''' Basic text cleaning for English documents
    
    param doc (str): document to be cleaned
    param stop_words (str list): common words to be filtered out
    return clean_doc (str): cleaned document
    '''
    # remove special characters and white space. This filters out non-Latin languages!!
    doc = re.sub(r'[^a-zA-Z\s]', ' ', doc, re.IGNORECASE|re.ASCII)
    # remove single characters
    doc = re.sub(r'\b[a-zA-Z]\b', '', doc, re.IGNORECASE|re.ASCII)    
    # remove whitespace at beginning and end of string
    doc = doc.strip()
    # convert all characters to lowercase
    doc = doc.lower()
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    # remove stop words
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # recreate doc from filtered tokens
    clean_doc = ' '.join(filtered_tokens)
    return clean_doc

stop_words = nltk.corpus.stopwords.words('english')
stop_words.extend(['co', 'https', 'http', 'gmail', 'com', 'like', 'love'])

users['clean_desc'] = users['description'].map(lambda doc: normalize_doc(doc, stop_words) 
                                                if doc is not None
                                                else np.nan)

users[['description','clean_desc']].head()

,description,clean_desc
0,so para fotos de desenhos,para fotos de desenhos
1,Extreme sports calendar model. \nKeeping retro alive since 1988.,extreme sports calendar model keeping retro alive since
2,None,NaN
3,#Unity3d #Game #Developer,unity game developer
4,"Still believes in the warmth in interaction. Studied games in culture & learning at MIT in CMS, SM '16. Works at UW-Ex; personal account here. She/her.",still believes warmth interaction studied games culture learning mit cms sm works uw ex personal account


For now, only profile descriptions in English were analyzed. Language identification of short text is challenging, and the more common language identication libraries (e.g. langid) were inaccurately classifying certain profile descriptions. Facebook's FastText library offers an alternative that has shown relatively high accuracy with short text language classification (http://alexott.blogspot.com/2017/10/evaluating-fasttexts-models-for.html). 

English profiles were lemmatized using the NLP libary spaCy, completing the text preprocessing stage of the analysis pipeline. Lemmatization can take several minutes, so the updated DataFrame was saved to a csv file. 

In [169]:
import os
import fasttext
import spacy

nlp = spacy.load('en') #download 'small' version of english model

def lemmatize_doc(doc):
    '''Apply spaCy lemmatization to a document (str)'''
    
    doc = nlp(doc)
    doc = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in doc])
    return doc

# import and apply FastText model for language identification
idlang_path = 'data/external/fasttext_training_data/lid.176.bin'
idlang_model = fasttext.FastText.load_model(idlang_path)

users['lang_id'] = users['clean_desc'].map(lambda doc: idlang_model.predict(doc) 
                                            if doc == doc 
                                            else np.nan)

# only English profiles are lemmatized; empty profiles are also ignored
users['clean_desc_en'] = users.apply(lambda row: lemmatize_doc(row['clean_desc']) 
                                        if  row['lang_id'] == row['lang_id']
                                        and row['lang_id'][0][0] == '__label__en'
                                        and row['clean_desc'] == row['clean_desc'] 
                                        else np.nan,
                                        axis=1)

# Push processed data into SQLite DB. Profiles with non-English or empty descriptions were dropped
e = sa.create_engine('sqlite:///./data/processed/' + DB_NAME + '-clean.sqlite')

cleaned_tab = TAB_NAME + '_clean'
users[['id', 'description', 'clean_desc_en']].to_sql(cleaned_tab, e, if_exists='replace')

users[['description','clean_desc','clean_desc_en']].head()

,description,clean_desc,clean_desc_en
0,so para fotos de desenhos,para fotos de desenhos,NaN
1,Extreme sports calendar model. \nKeeping retro alive since 1988.,extreme sports calendar model keeping retro alive since,extreme sport calendar model keep retro alive since
2,None,NaN,NaN
3,#Unity3d #Game #Developer,unity game developer,unity game developer
4,"Still believes in the warmth in interaction. Studied games in culture & learning at MIT in CMS, SM '16. Works at UW-Ex; personal account here. She/her.",still believes warmth interaction studied games culture learning mit cms sm works uw ex personal account,still believe warmth interaction study game culture learn mit cms sm work uw ex personal account


### Clustering users based on their profile descriptions

After preprocessing, the profile descriptions were clustered via two approaches, K-modes and DBSCAN. 

#### K-modes clustering

Given the short length of a typical profile, a word will typically appear no more than once within a given profile description. *It would thus be more approriate to vectorize profile descriptions as binary categorical vectors, as opposed to numeric vectors.* Profile descriptions were count vectorized as binary categorical data, where a value of *1* would indicate that a given word was present in a given profile description.  


K-means is likely the most popular approach to clustering. However, its use is limited to numerical data. The K-modes algorithim was developed as an analog to K-means for categorical data (Huang 1997, 1998). Like K-means, the K-modes algorithim requires that the number of clusters be predetermined.  

The kmodes Python library was used to perform K-modes clustering. No optimal number of clusters was assumed a priori. Consequently, K-modes clustering for a a range of initial cluster numbers (2 through 20) was executed. 

In [171]:
from sklearn.feature_extraction.text import CountVectorizer
from kmodes.kmodes import KModes

DB_NAME = 'customer-segmentation-clean'
TAB_NAME = 'ustwo_followers_clean'
NUM_CLUSTERS_RNG = 16 #upper limit of cluster range to evaluate

# Pull processed data from SQLite DB
e = sa.create_engine('sqlite:///./data/processed/' + DB_NAME + '.sqlite')

#SQL query includes a WHERE statement that filters out NA values and empty strings
query = 'SELECT id, description, clean_desc_en FROM ' + TAB_NAME + ' WHERE clean_desc_en != \'\''
users = pd.read_sql_query(query, e)

cv = CountVectorizer(ngram_range=(1, 2), min_df=100, max_df=1.0,
                     stop_words=stop_words, binary=True)
cat_matrix = cv.fit_transform(users['clean_desc_en'])

# Connect to DB where cluster results are to be saved
DB_RESULTS_NAME = 'customer-segmentation-cluster'
e = sa.create_engine('sqlite:///./models/' + DB_RESULTS_NAME + '.sqlite')
TAB_RESULTS_PREFIX = 'ustwo_followers_k_'

for n_clusters in range(2, NUM_CLUSTERS_RNG):
    kmod = KModes(n_clusters=n_clusters, init='Huang', random_state=42, n_jobs=-1)
    y_pred = kmod.fit_predict(cat_matrix.toarray())
    users['cluster'] = kmod.labels_
    tab_name = TAB_RESULTS_PREFIX + str(n_clusters)
    users.to_sql(tab_name, e, if_exists='replace')
    print('k = ' + str(n_clusters) + ' clusters analyzed')

k = 2 clusters analyzed
k = 3 clusters analyzed
k = 4 clusters analyzed
k = 5 clusters analyzed
k = 6 clusters analyzed
k = 7 clusters analyzed
k = 8 clusters analyzed
k = 9 clusters analyzed
k = 10 clusters analyzed
k = 11 clusters analyzed
k = 12 clusters analyzed
k = 13 clusters analyzed
k = 14 clusters analyzed
k = 15 clusters analyzed


The Kmodes algorithim will force the data to be organized into the predefined number of clusters. Clusters are not guaranteed to be generated in a meaningful way. 

In order to gauge the distinctiveness of the clusters, a vocabulary list of the most common unigrams, bigrams, and trigrams within the profile descriptions of a given cluster are generated (*gen_vocab_list*) and ordered by freqeuncy of occurance. The degree of similiarity in the top vocabulary between two clusters is a measure of the of the distance between those clusters.

In [172]:
from collections import Counter

def gen_vocab_list(corpus):
    ''' Generates a vocab list of unigrams, bigrams, and trigrams found in the corpus
    
    param corpus (str list): list containing documents from which vocab list will be derived
    return vocab (str list): list of vocab words found in the corpus
    '''
    unigrams = [words for doc in corpus for words in doc.split()]
    ngrams = [bigram for doc in corpus for bigram in nltk.ngrams(doc.split(), 2)]
    bigrams = [token[0] + ' ' + token[1] for token in ngrams]
    ngrams = [bigram for doc in corpus for bigram in nltk.ngrams(doc.split(), 3)]
    trigrams = [token[0] + ' ' + token[1] + ' ' + token[2] for token in ngrams]
    vocab = unigrams + bigrams + trigrams
    return vocab

def print_cluster_results(df, col_clust, col_list, clust_rng, n_keywords=10, n_desc=5):
    '''Evaluate clustering results by printing the most popular vocab words within a cluster corpus, and the prevalence of those words in the cluster corpus 
    
    param df (DataFrame): contains columns with profile features and cluster assignment
    param col_clust (str): name of column containing cluster assignment
    param col_list (str list): name of column containing corpus to be analyzed
    param clust_rng (tuple): bounds of the range of clusters to be printed. Assumes integer increments of 1. 
    param n_keywords (int): number of top vocab words to be analyzed for prevalence in the corpus
    param n_desc (int): number of randomly sampled twitter profiles whose features listed in col_list are to be printed
    '''
    n_total = len(df) 
    
    for cluster in range(clust_rng[0], clust_rng[1]):
        corpus = df.loc[df[col_clust] == cluster]
        n_corpus = len(corpus)
        per_label = round(n_corpus/n_total*100) # percentage of documents assigned to the cluster label 
        vocab = gen_vocab_list(corpus[col_list[0]])
        print('Cluster # ' + str(cluster))
        print(str(n_corpus) + ' (' + str(per_label) + '%) users with valid description in this cluster')
        print(Counter(vocab).most_common(n_keywords))
        if n_desc > 0:
            print(corpus[col_list].sample(n=n_desc))
        print('\n')

Rundown of ALL results

In [173]:
# NUM_CLUSTER = 2

DB_NAME = 'customer-segmentation-cluster'
e = sa.create_engine('sqlite:///./models/' + DB_NAME + '.sqlite')

for n_clusters in range(2, NUM_CLUSTERS_RNG):

    TAB_NAME = 'ustwo_followers_k_' + str(n_clusters)
    query = 'SELECT id, clean_desc_en, cluster FROM ' + TAB_NAME + ' WHERE clean_desc_en != \'\''
    users = pd.read_sql_query(query, e)

    print('CLUSTER K = ' + str(n_clusters))
    print('\n')
    print_cluster_results(df=users, col_clust='cluster', col_list=['clean_desc_en'], clust_rng=(0,n_clusters))

CLUSTER K = 2


Cluster # 0
47166 (98%) users with valid description in this cluster
[('game', 10080), ('designer', 3933), ('make', 2676), ('developer', 2441), ('design', 2342), ('gamer', 2325), ('artist', 2257), ('video', 2101), ('play', 2036), ('follow', 1895)]
                                                                                                           clean_desc_en
3435   future software developer                                                                                        
36339  game designer year found i would ion storm keen wolfenstein doom rise triad anachronox dash game ar vr pico nerdo
37726  us survival horror game follow story four survivor fight life spread infection                                   
10988  sun sign sure rise                                                                                               
21462  smash glass proceeding wide open party wound                                                                     


Cluster 

24568  ceo founder bro universe ltd                                                                                    


CLUSTER K = 5


Cluster # 0
3365 (7%) users with valid description in this cluster
[('designer', 3469), ('graphic', 556), ('game', 489), ('graphic designer', 470), ('ux', 464), ('design', 356), ('product', 336), ('illustrator', 333), ('artist', 301), ('ui', 278)]
                                                                          clean_desc_en
37222  director editor motion designer                                                 
33524  hobbyist programmer designer computer science uottawa                           
35668  game sound designer credit get even lego marvel super hero lego dc super villain
13171  interaction designer motion true passion pich de susan sarandon femininja       
46879  essentially shoe coffee lcfc tall graphic designer know                         


Cluster # 1
740 (2%) users with valid description in this cluster
[('stuff', 782), (

7624   designer day klimatosse night                                                                                        


Cluster # 4
485 (1%) users with valid description in this cluster
[('hi', 487), ('game', 111), ('name', 84), ('follow', 67), ('hi name', 63), ('play', 56), ('youtube', 55), ('video', 51), ('make', 45), ('i', 43)]
                                                                                                                     clean_desc_en
27759  hi typical girly gamer play sim sim also play minecraft youtube channel thankful everything guy                            
20926  hi savanna video game mostly destiny follow twich crazygirl crazy savanna stream every day                                 
21686  hi name bryce ilike play video game xbox keep rigid                                                                        
6104   hi i be terra socialist feminist bird parent sex positive vulture culture furry artist aspire chef tedpartyclothe snarlbaby
17860

32004  artist gamer writer many thank make hobby medium industry possible especially game devs                                 


CLUSTER K = 8


Cluster # 0
1428 (3%) users with valid description in this cluster
[('game', 1706), ('video', 1490), ('video game', 1341), ('play', 305), ('play video', 201), ('play video game', 197), ('make', 150), ('music', 116), ('youtube', 98), ('developer', 82)]
                                                                                 clean_desc_en
38660  fun play video game                                                                    
13936  designer artist also make video game                                                   
24394  youtuber gamer play game video freetime let get                                        
21647  attorney ny nj go ru video game film tv music entertainment law make something talk    
28239  read probably already follow go ahead check youtube channel motorcycle video game stuff


Cluster # 1
7183 (15%) users w

45644  diver violinist architect writer gamer instagram stringbandicoot                                            


Cluster # 1
196 (0%) users with valid description in this cluster
[('game', 239), ('programmer', 201), ('designer', 62), ('game programmer', 50), ('developer', 48), ('game designer', 39), ('game developer', 38), ('work', 30), ('artist', 27), ('make', 26)]
                                                                                    clean_desc_en
34221  game programmer game designer tech enthusiast                                             
13126  game dev review interest prefer twitter tumblr senior ui programmer ubisoft massive craigw
45738  game producer programmer web design draw game design reading writing etc                  
36286  programmer seahawk fan game news update game dev                                          
46991  game developer freelancer programmer                                                      


Cluster # 2
2442 (5%) users with val

[('artist', 1300), ('designer', 152), ('illustrator', 133), ('art', 123), ('concept', 80), ('concept artist', 74), ('animator', 73), ('writer', 65), ('freelance', 63), ('artist illustrator', 62)]
                                                            clean_desc_en
47726  artist founder alucina                                            
4396   designer illustrator photographer artist croteam                  
5901   pixel artist pixel daily gamedev twitchemoteartist dooz plwwcfxtmv
9662   fr en artist super slow make kind content                         
5585   character artist michigan bad adult pwhite                        


CLUSTER K = 11


Cluster # 0
41754 (87%) users with valid description in this cluster
[('game', 8430), ('designer', 3601), ('developer', 2250), ('make', 2228), ('gamer', 2070), ('design', 1949), ('artist', 1920), ('video', 1783), ('play', 1696), ('follow', 1565)]
                                                                                         clean

2618   freelance artist indie game dev work starrysummitgo ko fi jxtspcexip                                                      


Cluster # 6
515 (1%) users with valid description in this cluster
[('try', 522), ('game', 125), ('make', 114), ('try make', 76), ('get', 72), ('play', 45), ('try get', 45), ('life', 43), ('thing', 42), ('video', 42)]
                                                                              clean_desc_en
35450  quiet try musical video game lover                                                  
34381  gamer try get foot world gaming entertainment come check youtube euamnogpcd         
35488  sex positive lewd grill occasional gamer try avoid drama fun nice people follow back
38381  design studio middle east africa try make change design                             
21432  gamer play cod battlefield trickshotter try hard play snd domination                


Cluster # 7
7294 (15%) users with valid description in this cluster
[('game', 8603), ('video', 12

[('founder', 802), ('designer', 105), ('director', 82), ('ceo', 74), ('developer', 60), ('creative', 53), ('digital', 42), ('founder ceo', 41), ('artist', 36), ('product', 36)]
                                                                                                                             clean_desc_en
37590  logical creation xbox build team founder motra clutch flipsy                                                                       
40293  io dev founder air matter app ok qcbbc                                                                                             
32284  growth bd tokentax founder coingamma former mobile pm aaptiv iac nba trade derivative barclay lehmanbro learn wharton baruchcollege
15387  founder creative director phntmldn                                                                                                 
42352  digital director studiorotate luxury ecommerce agency founder harkable                                                   

46744  freelance plannery type person silly side project mostly make pixel trust                                                         


Cluster # 8
551 (1%) users with valid description in this cluster
[('channel', 590), ('youtube', 582), ('youtube channel', 490), ('check', 155), ('subscribe', 138), ('gaming', 103), ('game', 91), ('check youtube', 88), ('check youtube channel', 86), ('subscribe youtube', 67)]
                                                                     clean_desc_en
18386  youtube vlog channel zachary ritter vlog                                   
20647  subscribe youtube channel eastcoastkingyt                                  
17474  owner talkbox game youtube channel new scene anxious jump foot wait swim   
16998  sup guy mistixx competitive call duty gamer check youtube channel znyy voon
17084  hd clan youtube channel ta hd twitch stream ta hd youtube link             


Cluster # 9
350 (1%) users with valid description in this cluster
[('man', 368), ('

29251  marketing director svdp power baked good coffee pursuit micah                                                         


Cluster # 12
251 (1%) users with valid description in this cluster
[('social', 262), ('medium', 125), ('social medium', 114), ('marketing', 23), ('design', 20), ('digital', 17), ('music', 17), ('manager', 16), ('tweet', 15), ('gamer', 14)]
                                                                                                                                   clean_desc_en
1112   nerd word monkey husband dad obtainer rare antiquity film critic bbcoxford social tzone twilightzoneldn londoncritic author vintagegeek  
43141  netizen singularitarian marketing gadfly semi colon linux dogecoin maximalist fan confluence nuclear doomsday georgebaily mastodon social
31352  host content creator social medium manager hc                                                                                            
33059  social medium mgr uploadvr event thekoalition

Surveying the cluster results, common themes consistently arise. Gaming is clearly the most common interest among Monument Valley Twitter followers. Professions associated with the industry are also fairly popular, e.g. game designers, graphic designers, and UX designers. A few notable examples include the following:
 - *K=8, Cluster #3*: video game enthusiasts that stream on YouTube
 - *K=9, Cluster #8*: game developers, particularly indie studios
 - *K=10, Cluster #9*: creatives, e.g. graphic designers, artists, illustrators.

Of course, not all clusters are informative. For example, *K=6, Cluster #4* is primarily characterized by descriptions that include the word "hi". 

#### DBSCAN clustering

DBSCAN (density-based spatial clustering of applications with noise) is a density-based clustering algorithim. Unlike K-modes, which will force each data point to be assigned to a cluster, DBSCAN will mark data in low-density regions as outliers. 

Descriptions were vectorized by normalized term frequency. 

In [115]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import DBSCAN

DB_NAME = 'customer-segmentation-clean'
TAB_NAME = 'ustwo_followers_clean'

# Pull processed data from SQLite DB
e = sa.create_engine('sqlite:///./data/processed/' + DB_NAME + '.sqlite')

#SQL query includes a WHERE statement that filters out NA values and empty strings
query = 'SELECT id, clean_desc_en FROM ' + TAB_NAME + ' WHERE clean_desc_en != \'\''
users = pd.read_sql_query(query, e)
N_DESC = 10
N_KEYWORDS = 10

tf = TfidfVectorizer(use_idf=False)
tf_matrix = tf.fit_transform(users['clean_desc_en']) #tf is normalized as opposed to cv

dbscan = DBSCAN(eps=1.2, min_samples=1000)

y_pred = dbscan.fit_predict(tf_matrix)
users['cluster'] = dbscan.labels_

# Push processed data into SQLite DB. Profiles with non-English or empty descriptions were dropped
DB_RESULTS_NAME = 'customer-segmentation-cluster'
TAB_RESULTS_NAME = 'ustwo_followers_dbscan'
e = sa.create_engine('sqlite:///./models/' + DB_RESULTS_NAME + '.sqlite')
users.to_sql(TAB_RESULTS_NAME, e, if_exists='replace')

# Print results
n_clusters = len(np.unique(dbscan.labels_))
clust_rng = (-1, n_clusters-1) # offset by -1 since DBSCAN's first cluster is #-1
print_cluster_results(df=users, col_clust='cluster', col_list=['clean_desc_en'], clust_rng=clust_rng)

Cluster # -1
14653 (30%) users with valid description in this cluster
[('world', 391), ('founder', 381), ('work', 380), ('tweet', 337), ('get', 328), ('new', 327), ('fan', 316), ('tech', 315), ('team', 303), ('good', 299)]
                                                                                                                    clean_desc_en
39787  host free pay tourney xb ps let us know want pay tourney payout enjoy                                                     
7004   city bird always look sea geek philanthropist animallover romantic                                                        
29570  recruit analytic passionate datum programming tech innovation always happy talk datum feel free call                      
34900  write code storage start also computer vision gamedev illustration love explore stuff film graphic culture ex ce grad sjsu
11469  model imd tcm la model teresa imdmodele booking ig way poppin brooke bonnell                                            

DBSCAN determined that 70% of the cleaned Twitter descriptions were similiar enough to form a distint cluster (label = 0), while the remaining 30% were classified as outliers (label = -1). None of the top ten vocabulary words were shared between the Cluster #0 and "Cluster" #-1, suggesting good separation of the classes. 

The results corroborate those found via K-modes clustering. Based on the top vocabulary words of Cluster #0, a significant portion of USTWO Games Twitter followers are interested in gaming. This population seems to include video game fans, as well as professional or hobbyist creators: Half of the top ten vocabulary words are associated with video game design, art, and development. 

### Clustering users based on their friends

Clustering based on profile description has allowed us to identify useful customer segments: gamers, video game creators, developers, and Twitch/YouTube streamers. However, this approache has few limitations. Profile descriptions are not mandatory, and many Twitter users choose to leave the field blank (x% of USTWO followers did not have a completed description). Profiles that do contain a description may not contain relevant or useful information. Finally, this approach is biased towards professionals, or individuals with products or services to sell or advertise: these users have an incentive to create a profile description that is complete, informative, adn accurate. 

Another approach to customer segmentation is by clustering Twitter users based on the accounts that they follow (termed 'friends'). Presumably, users will only follow friends whose content they want to see: a user's friends are a reflection of their interests.

The user ID of the friends of USTWO Game followers were collected. K-modes categorical clustering was performed to determine if users could be clustered based on the accounts they followed. 

Due to the rate limitations of the Twitter API, collecting the friend IDs of every USTO Games follower would be time consuming. Stratified sampling based on the DBSCAN clustering results was performed to reduce the sample size to 10% (x users) 

In [142]:
# Get profiles that were clustered
DB_NAME = 'customer-segmentation-cluster'
TAB_NAME = 'ustwo_followers_dbscan'
e = sa.create_engine('sqlite:///./models/' + DB_NAME + '.sqlite')
query = 'SELECT id, clean_desc_en, cluster FROM ' + TAB_NAME
users_clustered = pd.read_sql_query(query, e)

# Get profiles that were not eligible for clustering (empty or not English)
DB_NAME = 'customer-segmentation-clean'
TAB_NAME = 'ustwo_followers_clean'
e = sa.create_engine('sqlite:///./data/processed/' + DB_NAME + '.sqlite')
query = 'SELECT id, clean_desc_en FROM ' + TAB_NAME + ' WHERE clean_desc_en == \'\' OR clean_desc_en IS NULL'
users_empty = pd.read_sql_query(query, e)
users_empty['cluster'] = -2 # create new 'cluster' feature, where all profiles assigned to same 'cluster'

# Combine dataframes
users = users_clustered.append(users_empty, ignore_index=True)

# Stratified sampling profiles by cluster assignment, pushing results to database
TAB_SAMPLES_NAME = 'ustwo_followers_sampled'
sampled_users = users.groupby('cluster').apply(lambda x: x.sample(frac=0.1, random_state=42))
sampled_users.reset_index(drop=True).to_sql(TAB_SAMPLES_NAME, e, if_exists='replace')


In [148]:
DB_NAME = 'customer-segmentation'
e = sa.create_engine('sqlite:///./data/interim/' + DB_NAME + '.sqlite')

query = '''
-- returns a list of friends (by screen name) that are followed by a given user
SELECT 
	user_id, GROUP_CONCAT(screen_name, " ") AS friends_list
FROM (
	-- only interested in friends that are followed by at least 75 users
	SELECT 
		user_id, friend_id
	FROM 
		ustwo_sampled_friends_ids
	WHERE
		friend_id
	IN (
		SELECT 
			friend_id
		FROM (
			-- filter out redundant user-friend id pairs
			SELECT DISTINCT
				user_id, friend_id
			FROM 
				ustwo_sampled_friends_ids
			-- filter out USTWO user id, since it will have no value in clustering since everyone follows the account
			WHERE 
				friend_id != 899902687
		)
		GROUP BY 
			friend_id
		HAVING
			COUNT(user_id) >= 75
	)
) AS tab
LEFT JOIN
	ustwo_sampled_fol_friends
ON 
	tab.friend_id == ustwo_sampled_fol_friends.id
GROUP BY 
	user_id
'''

users = pd.read_sql_query(query, e)

Setting the number of clusters to K=3 generated to most interesting clusters. The results are presented below. 

In [150]:
cv = CountVectorizer(min_df=1, max_df=1.0, binary=True)
cat_matrix = cv.fit_transform(users['friends_list'])

NUM_CLUSTERS_RNG = 6

# Connect to DB where cluster results are to be saved
DB_RESULTS_NAME = 'customer-segmentation-cluster'
e = sa.create_engine('sqlite:///./models/' + DB_RESULTS_NAME + '.sqlite')
TAB_RESULTS_PREFIX = 'ustwo_fol_friends_k_'

for n_clusters in range(2, NUM_CLUSTERS_RNG):
    kmod = KModes(n_clusters=n_clusters, init='Huang', random_state=42, n_jobs=-1)
    y_pred = kmod.fit_predict(cat_matrix.toarray())
    users['cluster'] = kmod.labels_
    tab_name = TAB_RESULTS_PREFIX + str(n_clusters)
    users.to_sql(tab_name, e, if_exists='replace')
    

In [160]:

DB_NAME = 'customer-segmentation-cluster'
e = sa.create_engine('sqlite:///./models/' + DB_NAME + '.sqlite')

for n_clusters in range(2, NUM_CLUSTERS_RNG):

    TAB_NAME = 'ustwo_fol_friends_k_' + str(n_clusters)
    query = 'SELECT user_id, friends_list, cluster FROM ' + TAB_NAME
    users = pd.read_sql_query(query, e)

    print('CLUSTER K = ' + str(n_clusters))
    print('\n')
    print_cluster_results(df=users, col_clust='cluster', col_list=['friends_list'],
                          clust_rng=(0,n_clusters), n_keywords=50, n_desc=0)

CLUSTER K = 2


Cluster # 0
6292 (53%) users with valid description in this cluster
[('RockstarGames', 5627), ('Ubisoft', 5593), ('PlayStation', 5568), ('Xbox', 5567), ('IGN', 5567), ('Kotaku', 5552), ('GameSpot', 5544), ('Twitch', 5535), ('TimOfLegend', 5533), ('giantbomb', 5532), ('engadgetgaming', 5529), ('HIDEO_KOJIMA_EN', 5507), ('NintendoAmerica', 5500), ('notch', 5494), ('Polygon', 5489), ('GamesRadar', 5488), ('gamasutra', 5488), ('SupergiantGames', 5483), ('PocketGamer', 5481), ('patrickklepek', 5476), ('fullbright', 5472), ('ID_AA_Carmack', 5465), ('telltalegames', 5460), ('2K', 5449), ('EAMobile', 5435), ('pgbiz', 5423), ('GooglePlay', 5422), ('GamesBeat', 5422), ('AppStore', 5404), ('leighalexander', 5401), ('acarboni', 5394), ('Quinns108', 5392), ('Pentadact', 5392), ('WH1SKI', 5390), ('levine', 5382), ('popcap', 5352), ('bfod', 5349), ('MaxTemkin', 5282), ('tinybop', 4542), ('tha_rami', 3868), ('brandonnn', 3825), ('chrisremo', 3816), ('br', 3778), ('stephentotilo', 3777)

In [93]:
# %% K-modes clustering, show top keywords in each cluster

NUM_CLUSTER = 3
N_KEYWORDS = 50
N_DESC = 0

PATH = os.path.normpath('c:\\Users\\Vincent\\Game-Revenant\\Shadows\\ustwo_sampled_friend_fol_cluster')
FILE_PREFIX = 'kmode_idlimit-75_'
FULL_PATH = os.path.join(PATH, FILE_PREFIX + str(NUM_CLUSTER) + '.csv')
results = pd.read_csv(FULL_PATH).dropna()
n_total = len(results)

for cluster in range(0, NUM_CLUSTER):
    corpus = results.loc[results['cluster'] == cluster]
    n_corpus = len(corpus)
    vocab = gen_vocab_list(corpus['friend_name'])
    print('Cluster: ' + str(cluster))
    print(str(n_corpus) + ' (' + str(round(n_corpus/n_total*100)) + '%) users with valid description in this cluster')
    print(Counter(vocab).most_common(N_KEYWORDS))
#    print(corpus[['description','keywords']].sample(n=N_DESC))
    print('\n')

Cluster: 0
4500 (41%) users with valid description in this cluster
[('BarackObama', 1708), ('elonmusk', 1590), ('NASA', 1299), ('PlayStation', 1276), ('BillGates', 1170), ('RockstarGames', 1132), ('Twitter', 1112), ('steam_games', 1073), ('YouTube', 1072), ('Ubisoft', 998), ('NintendoAmerica', 979), ('IGN', 953), ('Xbox', 946), ('HIDEO_KOJIMA_EN', 889), ('AppStore', 886), ('Polygon', 860), ('GooglePlay', 813), ('Twitch', 791), ('Kotaku', 788), ('TheEllenShow', 769), ('gamasutra', 758), ('tha_rami', 749), ('jimmyfallon', 735), ('TimOfLegend', 725), ('notch', 718), ('GameSpot', 671), ('SupergiantGames', 654), ('ID_AA_Carmack', 633), ('telltalegames', 601), ('bethesda', 501), ('2K', 491), ('femfreq', 427), ('PocketGamer', 408), ('engadgetgaming', 397), ('toucharcade', 385), ('helvetica', 377), ('GamesRadar', 368), ('fullbright', 367), ('bfod', 349), ('leighalexander', 345), ('majornelson', 344), ('levine', 342), ('CallofDuty', 341), ('popcap', 327), ('br', 318), ('brandonnn', 315), ('chri

The overlap in the top 50 accounts between clusters was minimal, suggesting good separation. 

Cluster \#2 is comprised of users that are interested in mainstream gaming, as fans or consumers. Top friends include large game publishers (Rockstar Games, Ubisoft), gaming consoles (Xbox, Playstation), gaming content mediums (Twitch), and publications (IGN, Polygon). 

Cluster \#1 is composed of users that are interested in indie video games (i.e. smallar video game creators). The accounts listed are primarily associated with individual video game developers or creators. Consultation with a domain expert (Frank DiCola, founder/CEO of Game Revenant and lead designer of Shadows) revealed that these users tend to be very active on Twitter, and will post content that is not strictly related to gaming (e.g. social issues, politics). 

The accounts in Cluster \#0 do not have a clear theme. A number of accounts tend to be generally popular, and have a very high number of followers (e.g. Barack Obama, Elon Musk). Another indication is that ratio between the users that follow the top accounts and the total number of users in the cluster is low (<0.5). 

## Business strategy recommendations

The following demographics should be prioritized in ad targeting

 - Video game fans and enthusiasts
 - Video game professionals
 - Indie game fans and professionals
 
Facebook and Twitter differ in their ad targeting capabilities. Recommendations will be specific to what each platform offers. 

### Recommendations for Facebook ads

Facebook allows targeting based on it users' profession and interests. Facebook-specific recommendations are as follows:
1. **Gaming professionals**. Users can be targeted based on their profession. Based on the K-modes analysis of @ustogames follower descriptions, professionals such as game designers, graphic designers, UX designers, illustrators and other creatives should be specifically targeted. 
2. **"Mainstream" video game fans**. Based on Cluster \#2 of the DBSCAN analysis of @ustogames follower friends, Facebook users who have expressed interest in popular gaming studios (Ubisoft, Rockstar Games), platforms (Xbox, Playstation), and media (Gamespot, Twitch) should be specifically targeted. 
3. **Indie video game fans**. Based on clustering analysis on @ustogames follower descriptions and friends, smaller independent gaming studios, developers, and media should specifically be targted.

A/B testing can be implemented to determine if any particular demographic is especially receptive to WSS advertising. 

### Recommendations for Twitter ads

Follower targeting is available on Twitter. Twitter users similiar to the followers of a list of usernames will be targeted. From the results of the DBSCAN analysis of @ustwo user friends, two user name lists can be generated, one for the usernames in Cluster \#1 and another for those in Cluster \#2. 

Follower targeting can also be performed with @ustogames and other similiar mobile premium game Twitter accounts.

Finally, keyword targeting is also available on Twitter. The keywords recommended for Facebook targeting can be used similarily. 